# Test Results Review Report

This is supposed to help create the milestone QA acceptance report. It is an interactive Jupyter document (or a static view created by [Weave](https://github.com/mpastell/Weave.jl/)) containing Julia code segments and their output. It is part of the JDP project which aims to create an *easily accessible* system for exploring test results and automating *arbitrary* workflows. Notebooks such as these are intended to provide an easy starting point for engineers and other technical users to create their own reports, possibly just by tweaking the existing ones.

If you are viewing the static output of this report and want to modify it then visit [gitlab.suse.de/richiejp/jdp](https://gitlab.suse.de/richiejp/jdp) for instructions on how to install and run JDP.

Obviously you can also access the library from a REPL or use it in a traditional script or application, but Jupyter provides a nice, persistent, graphical environment. I won't discuss how to use Jupyter in this notebook (just click on help at the top), but will heavily annotate the code.

This version of the report uses `DataFrames` to represent the data.

## Getting more help

Julia is self documenting.

In [ ]:
@doc Int
# hint: try @doc @doc

Using the built in docs is often better than asking the internet.

## Setup

First we need to load the JDP library which does the heavy lifting; importing and transforming the test result data from OpenQA into something useable. Note that this assumes you started this notebook by running `julia src/notebook.jl`.

> NOTE: It is required that you run this cell before the code cells following it. However not all of the cells need to be executed in order.

You may see a bunch of horrible angry red text when running this. Unfortunately this could either be info messages or error messages from the logging system, Jupyter treats both to a red background.

In [ ]:
# Monitors library source files and recompiles them after most changes
import Revise

# Run the init script which will setup the JDP project if necessary
include("../src/init.jl")

# Bring DataFrame's _members_ into our namespace, so we can call them directly
using DataFrames
using Markdown

# Import some libraries from the JDP project
using JDP.Conf
using JDP.Trackers.OpenQA    # Contains functions for dealing with the OpenQA web API
using JDP.Trackers.Bugzilla  # Functions for accessing the Bugzilla API(s)
using JDP.Repository

# Weave.jl can pass arguments to this notebook to control its output
args = try
    WEAVE_ARGS
catch
    Dict()
end

In [ ]:
builds = get(args, "builds", ["189.1", "0039"])

> NOTE: Julia has a _very_ strong type system, but we can still assign variables like a dynamic language. For library code it is generally a good idea to explicitly state what types you are expecting, but in Notebook code we can just let the compiler guess the type.

Next we may download some new results for a given build or builds to our local cache. This usually takes a long time, hence why there is a local cache. The results are then loaded into a `DataFrame` object. Alternatively you can specify `Vector` as the second argument, in which case you will get an array of `TestResult` objects.

In [ ]:
# Get some job results from the openqa.suse.de (osd) OpenQA instance.
df = Repository.fetch(OpenQA.TestResult, DataFrame, "osd", OpenQA.RecentOrInterestingJobsDef)
md"There are now $(size(df, 1)) test results"

The function `describe` from the DataFrames package gives us some stats and information about the structure of the loaded data.

In [ ]:
describe(df, stats = [:nunique, :min, :max, :eltype])

Look at the pretty table! We can also display graphs which could be even more delightful. Unfortunately it is slightly less pretty if you are viewing this as a static page.

## Failed tests for build

Let's look at what tests failed for the given builds. First we need to filter out passed test results and results from other builds. Then we can group the results by test name and suit, amalgamating some of the columns to make the table easier to view. Filter is fairly simple, but the grouping is a bit more complex and there is a bit of Julia magic, see [Split-Apply-Combine](http://juliadata.github.io/DataFrames.jl/stable/man/split_apply_combine.html) for help.

> NOTE: Packages such as QUERY.jl allow one to use an SQL like syntax which is probably a lot easier to understand for most people. However I think I actually prefer using plain structs with map-filter-reduce and for-loops than either of these (See `notebooks/Propagate Bug Tags.ipynb`).

In [ ]:
# The syntax "var -> expr" is an anonymous function, strings starting with 'r' are regexs.
# In Julia you don't need to write 'return' (unless you want to return early), most 
# statements return whatever the value of the final expression is
fails = filter(r -> r.build in builds && occursin(r"failed", r[:result]), df)

# group by name then apply the function defined by `do r ...` to each group
# Putting `do r` after `by` is like writing `by(r -> ...`. i.e. `do r` defines a function
# and passes it as the first argument to `by`.
fails_by_name = by(fails, [:name, :suit, :flags]) do r
    # 'by' first groups the results by name and suit then passes each group to us in the variable 'r'
    # we then use 'r' to produce a new DataFrame containing a single row. We return the new DataFrames 
    # and `by` then combines them... at least I think that is what happpens.
    DataFrame(
        # We have to write [] otherwise DataFrame creates a multi-row result (because r.result is an array)
        result = [unique(r.result)],
        arch = [unique(r.arch)],
        # Three dots `...` 'splats' an array (or tuple) into multiple function arguments 
        # and `vcat` concatenates it's arguments together
        refs = [filter(br -> br.tracker.api == nothing || br.tracker.api.name != "OpenQA", 
                          vcat(r.refs...)) |> unique]
        # also, and don't panic if this is a little more difficult to understand, 
        # 'unique' removes duplicate elements from a collection
    )
end

md"""$(nrow(fails_by_name)) tests failed in $(join(builds, ", ", " and "))"""

In [ ]:
bigdisplay(data) = withenv("LINES" => 100) do
    display(data)
end

Even with the `bigdisplay` function above, we may have too many failures to display all at once, so let's try displaying failures for subsets of tests.

### LTP failures

Missing bug refs

In [ ]:
no_bugrefs_ltp = filter(fails_by_name) do r
    isempty(r.refs) && # Remove tests which already have bug refs
    r.suit[1] == "LTP" &&    # Only include LTP results
    r.name != "boot_ltp" &&  # Don't include boot_ltp and shutdown_ltp modules
    r.name != "shutdown_ltp" &&
    !("Public Cloud" in r.flags) # LTP tests on Public cloud are displayed separately
end

bigdisplay(no_bugrefs_ltp)

With bug refs

In [ ]:
has_bugrefs_ltp = filter(fails_by_name) do r
    !isempty(r.refs) && 
    r.suit[1] == "LTP" &&    # Only include LTP results
    r.name != "boot_ltp" &&  # Don't include boot_ltp and shutdown_ltp modules
    r.name != "shutdown_ltp" &&
    !("Public Cloud" in r.flags)
end

bigdisplay(has_bugrefs_ltp)

### FS test failures

Missing bug refs

In [ ]:
no_bugrefs_fs = filter(fails_by_name) do r
    length(r.refs) < 1 &&
    r.suit[1] == "fstests"
end

bigdisplay(no_bugrefs_fs)

With bug refs

In [ ]:
has_bugrefs_fs = filter(fails_by_name) do r
    !isempty(r.refs) && r.suit[1] == "fstests"
end

bigdisplay(has_bugrefs_fs)

### Public cloud failures

Missing bug refs

In [ ]:
no_bugrefs_pc = filter(fails_by_name) do r
    isempty(r.refs) && "Public Cloud" in r.flags
end

bigdisplay(no_bugrefs_pc)

With bug refs

In [ ]:
has_bugrefs_pc = filter(fails_by_name) do r
    !isempty(r.refs) && "Public Cloud" in r.flags
end

bigdisplay(has_bugrefs_pc)

## Test status differences

This shows a history of tests which have changed status recently.

!!! note This needs updating to automatically figure out the most recent builds.

In [ ]:
# the builds we want to include in the diff
buildsv = sort(["190.3", "205.7", "211.2", "212.1"]) 
builds = Set(buildsv)
ignore = Set(["enable_kdump", "install", "partition", "run", "generate_report",
        "boot_to_desktop", "boot_ltp", "shutdown_ltp", "install_ltp", "proc_sys_dump"])

receant = filter(df) do r
    r.build in builds &&
    !(r.name in ignore)
end

build_labels = [Symbol(build) for build in sort(buildsv)]
with_builds = by(receant, [:name, :suit, :machine, :flags]) do group
    # Remove duplicates; we can only have one result per build
    group = by(group, :build) do sgroup
        # If we have multiple results for one build then we take the most positive
        # because often a test will fail due to some unrelated error and we rerun the job
        DataFrame(
            result = if any(r -> r == "passed", sgroup.result)
                "passed"
            elseif any(r -> r == "failed", sgroup.result)
                "failed"
            else
                sgroup.result[1]
            end
        )
    end
    sort!(group, :build)
    
    results = String[]
    for i in 1:length(buildsv)
        # Fill in missing build results
        if i > length(group.build) || buildsv[i] != group.build[i]
            push!(results, "none")
        else
            push!(results, group.result[i])
        end
    end
    
    # return a table with a single row of results for each build
    DataFrame([[r] for r in results], build_labels)
end

# Remove tests where every build had the same result
build_diff = filter(with_builds) do r
    any(b -> b != r[build_labels[1]], (r[blabel] for blabel in build_labels[2:end]))
end

md"$(size(build_diff, 1)) of $(size(with_builds, 1)) tests had differing results"

In [ ]:
filter(build_diff) do r
    r.suit[1] == "LTP"
end

In [ ]:
filter(build_diff) do r
    r.suit == ["fstests", "xfs"]
end

In [ ]:
filter(build_diff) do r
    r.suit == ["fstests", "btrfs"]
end